In [2]:
!python3.6 -m pip install torch

     |████████████████████████████████| 734.6MB 106kB/s eta 0:00:01   |                                | 2.2MB 948kB/s eta 0:12:53     |▍                               | 9.8MB 9.7MB/s eta 0:01:15     |███▍                            | 79.0MB 3.6MB/s eta 0:03:02     |█████▏                          | 118.8MB 2.5MB/s eta 0:04:04     |███████████                     | 251.4MB 2.7MB/s eta 0:02:57     |████████████████                | 368.4MB 2.1MB/s eta 0:02:57     |████████████████▌               | 377.7MB 1.4MB/s eta 0:04:24
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#loaded_state = torch.load(model_path+seq_to_seq_test_model_fname,map_location='cuda:0')
G = Generator(32,256,512,32).eval().to(device)

g_checkpoint = torch.load('autovc.ckpt', map_location=device)
G.load_state_dict(g_checkpoint['model'])

##################################
dir_path = "/tf/notebooks/SKAJPAI/voice_style_transfer/cross_model_resources/"

emb_org = np.load(dir_path+"emb_org.npy")
emb_trg = np.load(dir_path+"emb_trg.npy")
mel_org = np.load(dir_path+"mel_org.npy")
#mel_trg = np.load(dir_path+"mel_trg.npy")

emb_org = torch.from_numpy(emb_org).to(device)
emb_trg = torch.from_numpy(emb_trg).to(device)

mel_org = np.transpose(mel_org, axes=(1,0))
mel_org, len_pad = pad_seq(mel_org)
uttr_org = torch.from_numpy(mel_org[np.newaxis, :, :]).to(device)
print(uttr_org.shape, emb_org.shape, emb_trg.shape)
        
with torch.no_grad():
    _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)

if len_pad == 0:
    uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
else:
    uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()

np.save("uttr_trg", uttr_trg)
###################################

metadata = pickle.load(open('metadata.pkl', "rb"))
print('metadata shape: ',np.asarray(metadata).shape)
spect_vc = []

for sbmt_i in metadata:
             
    x_org = sbmt_i[2]
    print("x_org without padding: ",x_org.shape)
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        print(uttr_org.shape, emb_org.shape, emb_trg.shape)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(uttr_trg, handle)          

torch.Size([1, 704, 80]) torch.Size([1, 256]) torch.Size([1, 256])
metadata shape:  (4, 3)
x_org without padding:  (90, 80)
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
x_org without padding:  (89, 80)
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
x_org without padding:  (75, 80)
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
torch.Size([1, 96, 80]) torch.Size([1, 256]) torch.Size([1, 256])
x_org without padd